# Pandas (2)

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

In [1]:
import pandas as pd

df = pd.read_csv('sp500hst.txt', header=None, 
                 names=["date", "ticker", "open", "high", "low", "close", "volume"])
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.dtypes

date      datetime64[ns]
ticker            object
open             float64
high             float64
low              float64
close            float64
volume             int64
dtype: object

2. Посчитайте количество уникальных цифр, которые используются каждой строке в столбце volume.

In [2]:
def n_digits(volume):
    return len(
        set(
            str(volume)
        )
    )
n_digits(1234), n_digits(1232)

(4, 3)

In [3]:
df["volume"].map(n_digits).head(2)

0    5
1    3
Name: volume, dtype: int64

3. Для каждой строки рассчитайте разность между значениями high и low, если индекс столбца нечетный, и разность между close и high в противном случае.

In [4]:
df.head(2)

def diff(row):
    if row.name % 2:
        return row["high"] - row["low"]
    else:
        return row["close"] - row["open"]

df.apply(diff, axis=1)

0        -0.0500
1         0.4100
2        -0.1600
3         0.4975
4         0.0400
           ...  
122569    0.5200
122570   -0.1300
122571    0.7100
122572   -1.4100
122573    1.0700
Length: 122574, dtype: float64

In [5]:
import numpy as np

np.where(
   df.index % 2 == 1,
   df["high"] - df["low"],
   df["close"] - df["open"] 
)

array([-0.05,  0.41, -0.16, ...,  0.71, -1.41,  1.07])

4. Рассчитайте суммарный объем торгов для для одинаковых значений тикеров.

In [6]:
df.groupby('ticker')[["open", "volume"]].max()

,open,volume
ticker,,
A,37.25,118162
AA,17.49,1558986
AAPL,277.75,666824
ABC,33.19,192412
ABT,56.23,185338
...,...,...
XTO,48.51,1250384
YHOO,18.67,864125
YUM,43.83,156401


In [7]:
df.groupby('ticker').filter(lambda group: group["open"].max() >= 100)

,date,ticker,open,high,low,close,volume
490,2009-08-21,AAPL,167.81,169.37,166.80,169.22,148597
491,2009-08-24,AAPL,170.00,170.71,168.27,169.06,145331
492,2009-08-25,AAPL,169.43,170.94,169.13,169.40,115840
493,2009-08-26,AAPL,168.94,169.55,166.76,167.41,108570
494,2009-08-27,AAPL,168.59,169.57,164.83,169.45,160421
...,...,...,...,...,...,...,...
118668,2010-08-13,WPO,347.95,351.71,337.85,343.48,1681
118669,2010-08-16,WPO,304.00,321.14,295.56,315.65,8774
118670,2010-08-17,WPO,324.59,336.58,319.74,327.63,3035
118671,2010-08-19,WPO,337.94,348.40,333.46,345.61,2197


5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

In [8]:
df2 = pd.read_csv('sp_data2.csv', header=None, 
                  names=["ticker", "name", "annual_income"],
                  sep=";")

In [9]:
df.merge(df2, how="inner", left_on="ticker", right_on="ticker")

,date,ticker,open,high,low,close,volume,name,annual_income
0,2009-08-21,A,25.60,25.6100,25.220,25.55,34758,Agilent Technologies,0.1%
1,2009-08-24,A,25.64,25.7400,25.330,25.50,22247,Agilent Technologies,0.1%
2,2009-08-25,A,25.50,25.7000,25.225,25.34,30891,Agilent Technologies,0.1%
3,2009-08-26,A,25.32,25.6425,25.145,25.48,33334,Agilent Technologies,0.1%
4,2009-08-27,A,25.50,25.5700,25.230,25.54,70176,Agilent Technologies,0.1%
...,...,...,...,...,...,...,...,...,...
82167,2010-08-13,ZION,20.17,20.4300,19.840,19.89,25193,Zions Bancorp,0.0%
82168,2010-08-16,ZION,19.81,19.9600,19.600,19.95,25914,Zions Bancorp,0.0%
82169,2010-08-17,ZION,20.07,20.4700,19.830,20.31,31717,Zions Bancorp,0.0%
82170,2010-08-19,ZION,19.83,20.0000,19.130,19.35,45935,Zions Bancorp,0.0%


## Лабораторная работа №2.2

__Данная работа является продолжением ЛР №2. Для начала работы загрузите таблицы (см. задание 1.1)__

In [10]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv', index_col = 0)

### Применение функций к pd.Series и pd.DataFrame

4.1 Напишите функцию, которая переводит минуты в формат "XhYm". Примените эту функцию к столбцу `minutes` таблицы `recipes` (без перезаписи исходного столбца) при помощи метода `map`.

In [11]:
def m_to_hm(m: int) -> str:
    return f'{m//60}h{m%60}m' 

assert m_to_hm(90) == "1h30m"
assert m_to_hm(10) == "0h10m"

In [12]:
recipes['minutes'].map(m_to_hm)

0        1h30m
1        0h10m
2        0h30m
3        0h45m
4        0h25m
         ...  
29995    1h20m
29996     4h0m
29997    1h15m
29998     1h0m
29999    0h29m
Name: minutes, Length: 30000, dtype: object

4.2 На основе таблицы `recipes` создайте таблицу, которая содержит только текстовые столбцы (используйте метод `select_dtypes`).  Примените к каждому элементу этой таблицы строковый метод `str.capitalize` при помощи метода `applymap`, не удаляя пропуски.

In [13]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'], format = '%Y-%m-%d')

In [14]:
recipes_object = recipes.select_dtypes(include = 'object')
recipes_object

,name,description
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,"these were so go, it surprised even me."
3,italian gut busters,my sister-in-law made these for us at a family...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...
29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,"this is a traditional fresh plum cake, thought..."
29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,zydeco soup,this is a delicious soup that i originally fou...


In [15]:
recipes_object.applymap(str.capitalize, na_action = 'ignore')

,name,description
0,George s at the cove black bean soup,An original recipe created by chef scott meska...
1,Healthy for them yogurt popsicles,My children and their friends ask for my homem...
2,I can t believe it s spinach,"These were so go, it surprised even me."
3,Italian gut busters,My sister-in-law made these for us at a family...
4,Love is in the air beef fondue sauces,I think a fondue is a very romantic casual din...
...,...,...
29995,Zurie s holey rustic olive and cheddar bread,This is based on a french recipe but i changed...
29996,Zwetschgenkuchen bavarian plum cake,"This is a traditional fresh plum cake, thought..."
29997,Zwiebelkuchen southwest german onion cake,This is a traditional late summer early fall s...
29998,Zydeco soup,This is a delicious soup that i originally fou...


4.3 Напишите функцию, которая принимает на вход серию `pd.Series` и для серий, содержащих текстовые данные, возвращает максимальную длину строк в ней, а для числовых серий возвращает минимальный элемент в этой серии. Примените данную функцию к каждому столбцу таблицы `recipes` при помощи метода `apply`.

In [16]:
def get_stats(x: pd.Series) -> int:
    if x.dtype == 'O':
        return x.str.len().max()
    elif x.dtype == 'int64' or x.dtype == 'datetime64[ns]' or x.dtype == 'float64':
        return x.min()   

assert get_stats(pd.Series(['a', 'bbbb', 'ccc'])) == 4
assert get_stats(pd.Series([1, 3, 2])) == 1

In [17]:
recipes.apply(get_stats)

name                               83
id                                 48
minutes                             0
contributor_id                   1530
submitted         1999-08-06 00:00:00
n_steps                           1.0
description                    6321.0
n_ingredients                     1.0
dtype: object

### Группировки таблиц `pd.DataFrame`

5.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [18]:
recipes_number = recipes.groupby('contributor_id')['name'].count()
recipes_number

contributor_id
1530            5
1533          186
1534           50
1535           40
1538            8
             ... 
2001968497      2
2002059754      1
2002234079      1
2002234259      1
2002247884      1
Name: name, Length: 8404, dtype: int64

In [36]:
recipes_number[recipes_number == recipes_number.max()].index

Int64Index([89831], dtype='int64', name='contributor_id')

5.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [20]:
reviews.groupby('recipe_id')['rating'].mean(numeric_only = False)

recipe_id
48        1.000000
55        4.750000
66        4.944444
91        4.750000
94        5.000000
            ...   
536547    5.000000
536610    0.000000
536728    4.000000
536729    4.750000
536747    0.000000
Name: rating, Length: 28100, dtype: float64

In [21]:
recipes.shape[0] - reviews.groupby('recipe_id')['rating'].count().shape[0]

1900

5.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [22]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'], format = '%Y-%m-%d')
recipes['year'] = recipes['submitted'].dt.year 
recipes.groupby('year')['name'].count()

year
1999     275
2000     104
2001     589
2002    2644
2003    2334
2004    2153
2005    3130
2006    3473
2007    4429
2008    4029
2009    2963
2010    1538
2011     922
2012     659
2013     490
2014     139
2015      42
2016      24
2017      39
2018      24
Name: name, dtype: int64

5.4 Напишите функцию, которая принимает на вход таблицу (аналогичную `recipes` по набору столбцов), и возвращает `True` в том случае, если в столбце `minutes` присутствуют только значения, меньшие либо равные 10. Сгруппируйте таблицу `recipes` по полю `contributor_id` и для каждого участника выясните, справедливо ли, что все его рецепты занимают не более 10 минут.

In [23]:
import numpy as np

In [24]:
def has_only_fast_recipes(x: pd.DataFrame) -> bool:
    not_ten_minutes = x['minutes'] > 10
    if not_ten_minutes.sum() == 0:
        return True
    else:
        return False
    
assert not has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 90, 1: 10},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)
assert has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 7, 1: 5},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)

In [25]:
recipes.groupby('contributor_id').apply(has_only_fast_recipes)

contributor_id
1530          False
1533          False
1534          False
1535          False
1538          False
              ...  
2001968497    False
2002059754    False
2002234079    False
2002234259    False
2002247884    False
Length: 8404, dtype: bool

### Объединение таблиц `pd.DataFrame`

6.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [26]:
recipes_reviews = pd.merge(recipes, reviews, left_on = 'contributor_id', right_on = 'user_id')
recipes_reviews.drop(['minutes', 'contributor_id', 'submitted', 'n_steps', 'description', 'n_ingredients', 'recipe_id', 'date', 'review'], 
                     axis = 1, inplace = True)
recipes_reviews = recipes_reviews.drop_duplicates()
recipes_reviews

,name,id,year,user_id,rating
0,george s at the cove black bean soup,44123,2002,35193,5
24,allegedly original cobb salad dressing,39189,2002,35193,5
48,basic cake gluten free,52100,2003,35193,5
72,basil garlic butter,32311,2002,35193,5
96,beer marinated shrimp,36315,2002,35193,5
...,...,...,...,...,...
1984095,zucchini carrot cupcakes,468621,2011,135887,2
1984119,zucchini lemon bread,120742,2005,212714,0
1984120,zucchini with serrano ham,162411,2006,152500,4
1984121,zucchini with serrano ham,162411,2006,152500,3


In [27]:
recipes[-(recipes['id'].isin(reviews['recipe_id']))].head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,year
26,pasta with shrimp eggplant,223349,50,452592,2007-04-18,7.0,a nice low carb way to feel like you are eatin...,9.0,2007
29,secret ingredient bbq meatloaf,342620,75,50969,2008-12-09,9.0,"i know what you might be thinking,'ho hum, ano...",10.0,2008
35,windy s sweet and sour meatballs,276594,50,341338,2008-01-05,6.0,whenever we had a potluck lunch at the office ...,14.0,2008
49,goulashy beef stew for the slow cooker,216068,430,446143,2007-03-10,NaN,noodle beef stew in the slow cooker...easy com...,13.0,2007
59,old bay grilled steak fries,306590,20,337736,2008-05-31,NaN,"found this in a magazine, yet to try, but look...",15.0,2008


In [28]:
recipes_reviews[recipes_reviews['id'] == 22349]

,name,id,year,user_id,rating


6.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [29]:
recipes_reviews_count = pd.merge(recipes, reviews, left_on = 'id', right_on = 'recipe_id', how='left')
recipes_reviews_count.drop(['recipe_id','minutes', 'contributor_id', 'submitted', 'n_steps', 'description', 'n_ingredients', 'date', 'rating', 'user_id'],
                           axis = 1, inplace = True)
recipes_reviews_count = recipes_reviews_count.groupby(['name', 'id']).count()
recipes_reviews_count

,,year,review
name,id,,
0 fat chunky watermelon salsa,176996,1,1
1 001 nights cake,111198,3,3
1 1 1 tempura batter,164636,13,13
1 2 3 4 cake orange,165438,2,2
1 2 3 bread,33246,2,2
...,...,...,...
zuppa by luisa,464576,3,3
zurie s holey rustic olive and cheddar bread,267661,4,4
zwetschgenkuchen bavarian plum cake,386977,2,2


In [30]:
recipes_reviews_count.reset_index(inplace=True)
recipes_reviews_count

,name,id,year,review
0,0 fat chunky watermelon salsa,176996,1,1
1,1 001 nights cake,111198,3,3
2,1 1 1 tempura batter,164636,13,13
3,1 2 3 4 cake orange,165438,2,2
4,1 2 3 bread,33246,2,2
...,...,...,...,...
29995,zuppa by luisa,464576,3,3
29996,zurie s holey rustic olive and cheddar bread,267661,4,4
29997,zwetschgenkuchen bavarian plum cake,386977,2,2
29998,zwiebelkuchen southwest german onion cake,103312,6,6


In [31]:
recipes_reviews_count = recipes_reviews_count[['id', 'name', 'review']]
recipes_reviews_count.columns = ['recipe_id', 'name', 'review_count']
recipes_reviews_count

,recipe_id,name,review_count
0,176996,0 fat chunky watermelon salsa,1
1,111198,1 001 nights cake,3
2,164636,1 1 1 tempura batter,13
3,165438,1 2 3 4 cake orange,2
4,33246,1 2 3 bread,2
...,...,...,...
29995,464576,zuppa by luisa,3
29996,267661,zurie s holey rustic olive and cheddar bread,4
29997,386977,zwetschgenkuchen bavarian plum cake,2
29998,103312,zwiebelkuchen southwest german onion cake,6


6.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [32]:
recipes_reviews_rating = pd.merge(recipes, reviews, left_on = 'id', right_on = 'recipe_id')
recipes_reviews_rating.drop(['minutes', 'contributor_id', 'n_steps', 'description', 'n_ingredients', 'recipe_id', 'date', 'review', 'user_id'], axis = 1, inplace = True)
recipes_reviews_rating['submitted'] = pd.to_datetime(recipes_reviews_rating['submitted'], format = '%Y-%m-%d')
recipes_reviews_rating['year'] = recipes_reviews_rating['submitted'].dt.year

In [33]:
recipes_reviews_rating.groupby('year')['rating'].mean().sort_values().index[0]

2017

### Сохранение таблиц `pd.DataFrame`

7.1 Отсортируйте результат выполнения задания 6.1 в порядке убывания величины столбца `id` и сохраните результаты в csv файл. 

In [34]:
recipes_reviews.sort_values('id', ascending = False).to_csv('recipes_reviews.csv', index=False)

7.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 6.1 и 6.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 6.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 6.2.

In [35]:
with pd.ExcelWriter('recipes_reviews_rating.xlsx') as writer:
    recipes_reviews.to_excel(writer, sheet_name="Рецепты с оценками")
    recipes_reviews_count.to_excel(writer, sheet_name="Количество отзывов по рецептам")